In [573]:
from Processor import Processor
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pandas as pd

proccesor = Processor(filename_tags='./../data/IAB_tags.csv',
                      filename_dataset='./../data/train_data_categories.csv')
proccesor.compile()

dataset = proccesor.dataset
tags = proccesor.tags_df

In [574]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

dataset['meta_info_vector'] = dataset['meta_info'].apply(
    lambda l: model.encode(l, convert_to_tensor=True).cpu().numpy())
tags['tag_vector'] = tags['full_name'].apply(lambda l: model.encode(l, convert_to_tensor=True).cpu().numpy())

C:\Users\rustamPROLife\PycharmProjects\hack\.venv\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
from sentence_transformers import SentenceTransformer


def get_vectors(information):
    info_vector = model.encode(information, convert_to_tensor=True).cpu().numpy()
    return info_vector


def get_predictions(faiss, vector, top_n=3):
    search_vectors = vector.astype('float32')
    faiss.normalize_L2(search_vectors)
    scores, predictions = index.search(search_vectors, top_n)
    for j, i in enumerate(predictions):
        print("SCORES", scores[j])
        print("PREDICTION_by_title", np.array(tags['full_name'].values.tolist())[predictions[j]])
        print("SAMPLE", df['popular_words'].to_list()[:k][j])
        print("ORIGINAL TAGS", df['tags'][j])
        print("\n")

In [576]:
vectors = np.array(list(tags['tag_vector'].values))

index = faiss.index_factory(vectors.shape[1], "Flat", faiss.METRIC_INNER_PRODUCT)
print(index.ntotal)
index.add(vectors)
print(index.ntotal)

0
610


In [577]:
topn = 3
k = 5

search_vectors = np.array(dataset['meta_info_vector'].to_list()[:k]).astype('float32')
faiss.normalize_L2(search_vectors)
scores, predictions = index.search(search_vectors, topn)
for j, i in enumerate(predictions):
    print("SCORES", scores[j])
    print("PREDICTION_by_title", np.array(tags['full_name'].values.tolist())[predictions[j]])
    print("SAMPLE", dataset['meta_info'].to_list()[:k][j])
    print("ORIGINAL TAGS", dataset['tags'][j])
    print("\n")

SCORES [2.3944273 2.1723883 2.0932121]
PREDICTION_by_title ['Телевидение: Драма' 'Телевидение: Детские каналы'
 'Телевидение: Научно-фантастические каналы']
SAMPLE Пацанский клининг. Шоу «ЧистоТачка» | Повелитель ночи | Выпуск 17 Тяпа и Егор бросили вызов нестареющему «повелителю ночи». Смогут ли они подарить ему статус «ЧистоТачки»?
ORIGINAL TAGS Массовая культура: Юмор и сатира


SCORES [2.3394017 2.2359781 2.1702814]
PREDICTION_by_title ['Спорт: Покер и профессиональные азартные игры' 'Спорт: Борьба'
 'Спорт: Боевые искусства']
SAMPLE СarJitsu. 3 сезон, 6 серия. Нарек Симонян vs Жека Секси CarJitsu — бои в формате POP MMA, где вместо ринга бойцы сражаются в салоне автомобиля. В этом выпуске самые опасные и беспощадные! Симонян Нарек «Пухляш» и Жека «Секси» Ушаков Узнайте, осталось ли что-то от машины, в которой они встретились! По вопросам рекламы пишите на email
ORIGINAL TAGS События и достопримечательности:  Спортивные события, Массовая культура: Юмор и сатира


SCORES [1.8708808 

In [578]:

topn = 2

sample_submission = pd.DataFrame(data=dataset['video_id'].to_list(), columns=['video_id'])
sample_submission['predicted_tags'] = np.nan
sample_submission['predicted_tags'] = sample_submission['predicted_tags'].astype('object')

for i, row in dataset.iterrows():
    scores, predictions = index.search(np.array([row['meta_info_vector']]), topn)
    index_i = sample_submission[sample_submission.video_id == row.video_id].index
    predicted_tags = np.array(tags['full_name'].values.tolist())[predictions[0][0]]
    sample_submission.at[index_i[0], 'predicted_tags'] = [predicted_tags]

In [579]:
sample_submission['predicted_tags'] = sample_submission['predicted_tags'].astype(str)
sample_submission.to_csv("sample_submission.csv", index_label=0)

In [580]:
dataset['tags'] = dataset['tags'].astype(str)

In [581]:
from metrics import find_iou_for_sample_submission

score = find_iou_for_sample_submission(pred_submission=sample_submission,
                                       true_submission=dataset)
print(score)

0.14613095238095244


## Популярные слова

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
import pymorphy2

In [599]:
new_proccesor = Processor(filename_tags='./../data/IAB_tags.csv',
                          filename_dataset='./../result.csv')
new_proccesor.compile()


In [600]:
df = new_proccesor.dataset
df = df[(df['popular_words'].notna()) & (df['tags'].notna())].reset_index(drop=True)
df

,video_id,title,description,tags,popular_words,clean_tags,cat_1,cat_2,cat_3,meta_info
0,f02709d3f6b2fbd265863ef538b13c0a,Смешная история | Выпуск 3,"Юный Михаил Ломоносов отправляется в Москву на повозке, используя современные технологии, Иван Грозный приступает к курсам онлайн-терапии, а Александр Сергеевич Пушкин пытается вспомнить, чем обидел возлюбленную. Новый выпуск «Смешной истории»! Где-то правда, где-то смешно!","Массовая культура, Изобразительное искусство","наш, жизнь, ждать, речесый, лен, давать, время, современный, ваш, живить, история, гдеть, стать, алло, который, великий, передавать, друг, заметить, селиться, меняться, здоровье, свидетельный, врач, смарт, часы, случиться, перебиться, минутка, пульс, 150, палозить, веудочка, подариться, тратить, успокоить, ребёнок, технология, настолько, казаться, реально, смешной, правда, смешно, 18й, век, хаилла, маносов, стремиться, выдающийся, учь, путь, успех, тяжёлый, долгий, дорога, село, холмагорев, москов, сын, увидеть, москуда, лист, мочь, забыть, пойдуся, лавтие, допущий, гести, дать, бог, деревня, узнать, твой, полика, кудряшка, заяваета, поехать, дущерат, следующий, воздух, полгода, самый, ближний, сидиться, брат, шутдора, обрат, угол, васаха, надай, младить, стоять, место, компуд, идти, сколько, маска, девять, надцитый","[Массовая культура, Изобразительное искусство]",Массовая культура,None,None,"Смешная история | Выпуск 3 Юный Михаил Ломоносов отправляется в Москву на повозке, используя современные технологии, Иван Грозный приступает к курсам онлайн-терапии, а Александр Сергеевич Пушкин пытается вспомнить, чем обидел возлюбленную. Новый выпуск «Смешной истории»! Где-то правда, где-то смешно!"
1,c037b00580b5fcd9b8b85f3c83b01591,Люди у которых клюет | Выпуск 22 | Крайняя рыбалка на жерлицы. Поиск щуки на пруду в марте,"Отправились на пруд половить щуку на жерлицы. Развели костёр и поели тушенку. Как ловить щуку в марте, смотрите в этом выпуске.",Спорт: Рыбалка,"знать, мочь, просто, вау, замереть, поставить, пока, ветер, холодный, очень, звук, сильно, салошлик, вообще, держать, динамичный, музыка, днимаять, решение, идти, далёкий, ковжержиться, ковжериться, посмотреть, чуковый, чапач, брать, 20, закрытие, сезон, пожерываться, достаточно, общий, проаналётный, рыбалка, сказать, изгонять, думать, жернец, прям, крайне, вроде, низко, температура, солнце, светить, сквозной, немного, подмороживать, лицо, возможно, брак, микрофон, задываета, третий, рочек, подписчик, сердиться, получиться, задолвалить, ситуация, смех, причинный, время, расстояние, получаться, 9, метр, 19, сантиметр, огжерлица, рлиться, слышный, характерный, пластик, облёт, воракать, голова, исподъесть, крустить, круто, давать, подождать, схватить, пробить, ропить, снег, пускай, лесклиный, натян, сшибить, уйти",[Спорт: Рыбалка],Спорт,Рыбалка,None,"Люди у которых клюет | Выпуск 22 | Крайняя рыбалка на жерлицы. Поиск щуки на пруду в марте Отправились на пруд половить щуку на жерлицы. Развели костёр и поели тушенку. Как ловить щуку в марте, смотрите в этом выпуске."
2,806e572c94adf5e4deb8041d8f094d91,ВЫПУСК №2 НЕУДОБНЫЙ СТЕНДАП,"Андрей Колмачевский, Кирилл Касумов и Настя Веневитина выступят возле вокзала, в трамвае и в салоне красоты. Кто унесёт все деньги и любовь случайных зрителей, а кто уйдёт ни с чем - смотрите в этом выпуске Неудобного Стендапа!",Массовая культура: Юмор и сатира,"спасибо, говорить, большой, делать, шутка, начинать, видеть, свой, село, встретить, знать, человек, тема, дил, нравиться, жанр, микрофон, стендап, место, вокзал, приятно, ваш, который, недавно, заходить, просто, следующий, гена, дать, очень, ребёнок, жериновский, комедия, глаз, ребята, идти, благодарить, пьпьпать, вечно, посиваета, детoria, бабушка, практичный, подарок, тип, служба, здравие, ублюдский, ген, надеживой, вамщий, любить, считать, общий, сон, баня, площадь, неадан, землера, реально, смешить, море, мешок, пополго, смотреть, помоему, мастер, простить, деньга, хотеть, обоступление, сказать, неудобный, шелфтект, колокий, р

In [601]:

nltk.download('stopwords')
russian_stopwords = set(stopwords.words("russian"))
morph = pymorphy2.MorphAnalyzer()


def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r"[^а-яА-Я]", ' ', text)
    words = text.split()
    lemmatized_words = []
    for word in words:
        if word not in russian_stopwords:
            lemmatized_word = morph.parse(word)[0].normal_form  # Лемматизация
            lemmatized_words.append(lemmatized_word)
    cleaned_text = " ".join(lemmatized_words)

    return cleaned_text

df['clean_meta_info'] = df['meta_info'].apply(clean_text)
df['check'] = df['clean_meta_info'] + ' ' + df['popular_words']


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rustamPROLife\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [602]:
df['meta_info_with_popular_words_vector'] = df['check'].apply(
    lambda l: model.encode(l, convert_to_tensor=True).cpu().numpy())

In [603]:
topn = 3
k = 5

search_vectors = np.array(df['meta_info_with_popular_words_vector'].to_list()[:k]).astype('float32')
faiss.normalize_L2(search_vectors)
scores, predictions = index.search(search_vectors, topn)
for j, i in enumerate(predictions):
    print("SCORES", scores[j])
    print("PREDICTION_by_title", np.array(tags['full_name'].values.tolist())[predictions[j]])
    print("SAMPLE", df['popular_words'].to_list()[:k][j])
    print("ORIGINAL TAGS", df['tags'][j])
    print("\n")

SCORES [2.3724785 2.23885   2.093571 ]
PREDICTION_by_title ['Массовая культура: Юмор и сатира' 'Телевидение: Комедийные каналы'
 'Телевидение: Драма']
SAMPLE наш, жизнь, ждать, речесый, лен, давать, время, современный, ваш, живить, история, гдеть, стать, алло, который, великий, передавать, друг, заметить, селиться, меняться, здоровье, свидетельный, врач, смарт, часы, случиться, перебиться, минутка, пульс, 150, палозить, веудочка, подариться, тратить, успокоить, ребёнок, технология, настолько, казаться, реально, смешной, правда, смешно, 18й, век, хаилла, маносов, стремиться, выдающийся, учь, путь, успех, тяжёлый, долгий, дорога, село, холмагорев, москов, сын, увидеть, москуда, лист, мочь, забыть, пойдуся, лавтие, допущий, гести, дать, бог, деревня, узнать, твой, полика, кудряшка, заяваета, поехать, дущерат, следующий, воздух, полгода, самый, ближний, сидиться, брат, шутдора, обрат, угол, васаха, надай, младить, стоять, место, компуд, идти, сколько, маска, девять, надцитый
ORIGINAL TAGS 

In [604]:
import pandas as pd

topn = 2

sample_submission = pd.DataFrame(data=df['video_id'].to_list(), columns=['video_id'])
sample_submission['predicted_tags'] = np.nan
sample_submission['predicted_tags'] = sample_submission['predicted_tags'].astype('object')

for i, row in df.iterrows():
    scores, predictions = index.search(np.array([row['meta_info_with_popular_words_vector']]), topn)
    index_i = sample_submission[sample_submission.video_id == row.video_id].index
    predicted_tags = np.array(tags['full_name'].values.tolist())[predictions[0][0]]
    sample_submission.at[index_i[0], 'predicted_tags'] = [predicted_tags]

In [605]:
sample_submission['predicted_tags'] = sample_submission['predicted_tags'].astype(str)
sample_submission.to_csv("sample_submission.csv", index_label=0)

In [606]:
df['tags'] = df['tags'].astype(str)

In [607]:
from metrics import find_iou_for_sample_submission

score = find_iou_for_sample_submission(pred_submission=sample_submission,
                                       true_submission=df)
print(score)

0.15664350846721684


In [ ]:
faiss.write_index(index, './../index_file.index')
model.save_pretrained(path='./../model')
tags.to_csv('./../preprocessed_tags.csv',index=False)